In [4]:
# store start time to get execution time of entire script
import time
start_time = time.time()

In [5]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import csv

df = pd.read_csv('../data/data_cleaned.csv')
df.head()

,Unnamed: 0,ID,State,City,agyaddr,xobsyr_0,Illicit_Days,Illicit_Cens,female,nonwhite,unemplmt_cd,prsatx_cd,gvsg_cd,CWSg_0_cd,dssg_0_cd,epsg_0_cd,adhdg_0_cd,cdsg_0_cd,cjsig_0_cd,lrig_0_cd,srig_0_cd,SESg_0_cd,r4ag_0_cd,SUDSy_0_cd,homeless_0_cd,ncar_cd,TRIg_0_cd,PYS9Sxg_cd,primsev_other,primsev_alcohol,primsev_amphetamines,primsev_cocaine,primsev_marijuana,primsev_opioids,B2a_0g
0,0,3819,AZ,Tucson,3130 E Broadway Blvd,2005,363,0,0,0,1,1,2,0,2,1,2,1,1,1,1,0,0,3,0,0,0,0,0,0,1,0,0,0,0
1,1,857,OH,Cleveland,1276 West Third St. #400,2005,365,0,0,0,0,0,2,0,1,1,1,1,0,1,2,0,1,1,0,1,0,0,0,1,0,0,0,0,0
2,2,929,OH,Cleveland,1276 West Third St. #400,2006,354,0,0,0,0,0,1,0,0,1,0,1,1,0,1,0,2,2,0,0,1,0,0,1,0,0,0,0,0
3,3,951,OH,Cleveland,1276 West Third St. #400,2006,365,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,2,2,0,0,1,0,0,0,0,0,1,0,0
4,4,1032,OH,Cleveland,1276 West Third St. #400,2006,365,0,0,0,0,0,2,0,1,1,1,1,0,2,1,0,2,2,0,1,1,0,0,0,0,0,1,0,0


In [6]:
df['Address'] = df['agyaddr'] + ', ' + df['City'] + ', ' + df['State']
addresses = list(df['Address'].unique())
len(addresses)

188

In [7]:
%%time

from opencage.geocoder import OpenCageGeocode
from tqdm.notebook import tqdm

# get longitude and latitude for each unique address
key = '2d4ad4cfcdc44021abfe1e347daa6dc7'
geocoder = OpenCageGeocode(key)

geocoded_addrs = []

for addr in tqdm(addresses):
    results = geocoder.geocode(addr)
    lat = results[0]['geometry']['lat']
    lng = results[0]['geometry']['lng']
    geocoded_addrs.append([addr, lat, lng])
    # print(addr + ' -> (' + str(lat) + ', ' + str(lon) + ')')


CPU times: user 4.54 s, sys: 590 ms, total: 5.13 s
Wall time: 3min 11s


In [8]:
geocoded_addrs_df = pd.DataFrame(geocoded_addrs, columns=['Address', 'lat', 'lng'])
geocoded_addrs_df.head()

,Address,lat,lng
0,"3130 E Broadway Blvd , T...",32.221465,-110.926070
1,"1276 West Third St. #400 , C...",41.501028,-81.697772
2,"100 Oakdale Campus - M319 , I...",41.708538,-91.592663
3,"1220 Bardstown Rd , L...",38.236398,-85.717815
4,"1111 Barranca Dr., Suite 800 , E...",31.747334,-106.335056


In [9]:
geocoded_addrs_df.isna().sum()

Address    0
lat        0
lng        0
dtype: int64

In [10]:
%%time

import urllib, json, requests
import numpy as np

# get block_fips code based on longitude and latitude
def api_call(row):
    lat = row['lat']
    # http://data.fcc.gov/api/block/find?format=json&latitude=28.35975&longitude=-81.421988&showall=true
    lng = row['lng']
    URL = "https://geo.fcc.gov/api/census/block/find?format=json&latitude=" + str(lat) + "&longitude=" + str(lng)
    with urllib.request.urlopen(URL) as url:
        data = json.loads(url.read().decode())
    try:
        return int(data['County']['FIPS']), int(data['Block']['FIPS']), data['State']['name']
    except TypeError: # response failed to get county fips
        return np.nan, np.nan, np.nan

geocoded_addrs_df['state_name'] = ''
for ind,row in tqdm(geocoded_addrs_df.iterrows(), total=geocoded_addrs_df.shape[0]):
    county_FIPS, block_FIPS, state_name = api_call(row)
    geocoded_addrs_df.set_value(ind, 'county_FIPS', county_FIPS)
    geocoded_addrs_df.set_value(ind, 'block_FIPS', block_FIPS)
    geocoded_addrs_df.set_value(ind, 'state_name', state_name)
    
geocoded_addrs_df.head()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead



CPU times: user 3.25 s, sys: 304 ms, total: 3.55 s
Wall time: 1min 21s


,Address,lat,lng,state_name,county_FIPS,block_FIPS
0,"3130 E Broadway Blvd , T...",32.221465,-110.926070,Arizona,4019.0,4.019002e+13
1,"1276 West Third St. #400 , C...",41.501028,-81.697772,Ohio,39035.0,3.903511e+14
2,"100 Oakdale Campus - M319 , I...",41.708538,-91.592663,Iowa,19103.0,1.910300e+14
3,"1220 Bardstown Rd , L...",38.236398,-85.717815,Kentucky,21111.0,2.111101e+14
4,"1111 Barranca Dr., Suite 800 , E...",31.747334,-106.335056,Texas,48141.0,4.814100e+14


In [11]:
geocoded_addrs_df.isna().sum() # how many nulls per column

Address        0
lat            0
lng            0
state_name     1
county_FIPS    1
block_FIPS     1
dtype: int64

In [12]:
# re-merge patient data and geocoded addresses by the address feature
df = pd.merge(df, geocoded_addrs_df, on=['Address'], how='left')

In [13]:
df = df.dropna() # remove any remaining null-valued rows

In [14]:
df.shape

(21896, 41)

In [15]:
df.head()

,Unnamed: 0,ID,State,City,agyaddr,xobsyr_0,Illicit_Days,Illicit_Cens,female,nonwhite,unemplmt_cd,prsatx_cd,gvsg_cd,CWSg_0_cd,dssg_0_cd,epsg_0_cd,adhdg_0_cd,cdsg_0_cd,cjsig_0_cd,lrig_0_cd,srig_0_cd,SESg_0_cd,r4ag_0_cd,SUDSy_0_cd,homeless_0_cd,ncar_cd,TRIg_0_cd,PYS9Sxg_cd,primsev_other,primsev_alcohol,primsev_amphetamines,primsev_cocaine,primsev_marijuana,primsev_opioids,B2a_0g,Address,lat,lng,state_name,county_FIPS,block_FIPS
0,0,3819,AZ,Tucson,3130 E Broadway Blvd,2005,363,0,0,0,1,1,2,0,2,1,2,1,1,1,1,0,0,3,0,0,0,0,0,0,1,0,0,0,0,"3130 E Broadway Blvd , T...",32.221465,-110.926070,Arizona,4019.0,4.019002e+13
1,1,857,OH,Cleveland,1276 West Third St. #400,2005,365,0,0,0,0,0,2,0,1,1,1,1,0,1,2,0,1,1,0,1,0,0,0,1,0,0,0,0,0,"1276 West Third St. #400 , C...",41.501028,-81.697772,Ohio,39035.0,3.903511e+14
2,2,929,OH,Cleveland,1276 West Third St. #400,2006,354,0,0,0,0,0,1,0,0,1,0,1,1,0,1,0,2,2,0,0,1,0,0,1,0,0,0,0,0,"1276 West Third St. #400 , C...",41.501028,-81.697772,Ohio,39035.0,3.903511e+14
3,3,951,OH,Cleveland,1276 West Third St. #400,2006,365,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,2,2,0,0,1,0,0,0,0,0,1,0,0,"1276 West Third St. #400 , C...",41.501028,-81.697772,Ohio,39035.0,3.903511e+14
4,4,1032,OH,Cleveland,1276 West Third St. #400,2006,365,0,0,0,0,0,2,0,1,1,1,1,0,2,1,0,2,2,0,1,1,0,0,0,0,0,1,0,0,"1276 West Third St. #400 , C...",41.501028,-81.697772,Ohio,39035.0,3.903511e+14


In [16]:
df.to_csv('../data/data_geocoded.csv')

In [17]:
# print out total notebook execution time
total_seconds = int(time.time() - start_time)
minutes = total_seconds // 60
seconds = total_seconds % 60
print("--- " + str(minutes) + " minutes " + str(seconds) + " seconds ---")

--- 4 minutes 34 seconds ---
